In [1]:
# UIB paths
import os
path = os.getcwd()

if path[1:5] == "Home":
    %run /Home/siv30/wad005/master/GRB/src/paths/uib.ipynb
    %run /Home/siv30/wad005/master/GRB/src/read_matlab_file.ipynb
else:
    %run /Users/andreas/phys/master/GRB/src/paths/uib.ipynb
    %run /Users/andreas/phys/master/GRB/src/read_matlab_file.ipynb

all UiB paths loaded
All MAC paths are set!
read_matlab_file.ipynb loaded successfully!


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, re, sys

from astropy.io import fits, ascii
from astropy.time import Time
from stingray import Lightcurve as LC
from stingray import CrossCorrelation
from dateutil import parser

import gbm
from gbm.data import TTE,RSP,Trigdat
from gbm.data.primitives import EventList
from gbm.binning.unbinned import bin_by_time
from gbm.plot import Lightcurve, Spectrum
from gbm.background import BackgroundFitter
from gbm.background.binned import Polynomial #working in ctime for example

/Users/andreas/phys/master/FERMI/venv/lib/python3.7/site-packages/stingray/largememory.py:26: UserWarning: Large Datasets may not be processed efficiently due to computational constraints
  "Large Datasets may not be processed efficiently due to "
/Users/andreas/phys/master/FERMI/venv/lib/python3.7/site-packages/gbm/plot/lal_post_subs.py:184: UserWarning: Basemap not installed. Some functionality not available.
  warnings.warn('Basemap not installed. Some functionality not available.')


# Notes
    - gbm.data.pha.PHA (pha) class is for count spectra
    - gbm.data.phaii.PHAII (phaii) class is for time history and spectra.
        can get spectra by calling this funcs: phaii.to_spectrum()
        
    - Cross corrolation is done with a ms resolution

In [ ]:
def crossCorrolate(tus_F=None, kw_filename=None, binsize=None, detector=None, plotting=False):
    """
    TODO: change input from tus_F to tte. Then create a phaii object with energy_range specified from detector_range variable --> lc_data, then LC object with bin_lo +binsize/2 and counts
    
    
    
    cross corrolation function between FERMI and KW. Function will itself bin the photon array and return the desired time lag to shift the data into KW ref frame.
    
    Add t_shift to FERMI lc to bring it into KW ref frame.
    
    Note:   Only the bins equal to binsize will be used in KW data. 
            tus_F converted to float32 and KW data converted to float16 for memory saving. 
            --> KW data only accurate down to 1 ms
    
    input:
    --------------------------------------
    tus_F <np.array>: tus array given in seconds. Converted to float32.
    kw_filename <str>: filename.
    binsize <float>: given in seconds
    detector_range <str>: "high" or "low". High assumes BGO. Corrolates best with G2 (mid) + G3 (high). Low assumes NaI. Corrolates best with G1(low) + G2(mid)
    plotting <bool>
    
    return:
    --------------------------------------
    tlagg <float>: given in seconds. Rounded to 6 sig figures
    if value is positive the FERMI lc must be shiftet right from T0_FERMI and T0 slides to the left
    elif value is negative the FERMI lc must be shifted left og T0_FERMI and T0 slides to the right
    
    t_shift must be added to fermi tte to match the T0 to KW
    """
    
    

    
    #KW handling------------------------------------------------------------------------------
    if str(int(binsize*1e3)) != kw_filename[kw_filename.find('_')+1:-6]:
        return print('filename and binsize is not correct')
    
    kw_file = kw_lc_path + kw_filename
    kw_df = pd.read_csv(kw_file,sep='\s+')
    kw_bin_lo, kw_bin_hi = kw_df["Ti"].to_numpy(dtype=np.float32), kw_df["Tf"].to_numpy(dtype=np.float32)

    kw_dt = np.round(np.diff((kw_bin_lo)),3)
    i_stop_kw = np.where(kw_dt>2*binsize)[0][0] #floating point error throwing off

    kw_bin_lo, kw_bin_hi = kw_bin_lo[:i_stop_kw], kw_bin_hi[:i_stop_kw]

    if detector == "BGO":
        G2, G3 = kw_df["G2"].to_numpy(dtype=np.float16)[:i_stop_kw], kw_df["G3"].to_numpy(dtype=np.float16)[:i_stop_kw]
        kw_counts = G2 + G3
    elif detector == "NaI":
        G1, G2, G3 = kw_df["G1"].to_numpy(dtype=np.float16)[:i_stop_kw], kw_df["G2"].to_numpy(dtype=np.float16)[:i_stop_kw], kw_df["G3"].to_numpy(dtype=np.float16)[:i_stop_kw]
        kw_counts = G1 + G2 + G3
    else:
        print("detector not set. For NaI choose 'low' and BGO choose 'high'")

    kw_lc = LC(time=kw_bin_lo + binsize/2, counts=kw_counts,skip_checks=True,dt=binsize)
                    
    #FERMI handling-------------------------
    tus_F = np.float64(tus_F)
    kw_first_t, kw_last_t = find_nearest(tus_F,kw_bin_lo[0]), find_nearest(tus_F,kw_bin_hi[-1])
    tus_F = tus_F[kw_first_t:kw_last_t]    
    fermi_lc = LC.make_lightcurve(tus_F, dt=binsize)
    
    # Cross corrolating and shifting Fermi
    #The first Lightcurve passed is the channel of interest or interest band,
    #and the second Lightcurve passed is the reference band.
    
    CC = CrossCorrelation(kw_lc,fermi_lc,mode="full")
    tlagg = np.round(CC.time_shift,6)
    fermi_lc_shifted = fermi_lc.shift(tlagg) #CC.time_shift
    
    #Plotting -----------------------------
    if plotting == True:
        kw_lc.plot(title="KW lc",axis=(-0.1,0.1,0,70))
        fermi_lc_shifted.plot(title="FERMI lc shifted",axis=(-0.1,0.1,0,70))
        fermi_lc.plot(title="FERMI lc",axis=(-0.1,0.1,0,70))
        CC.plot(labels = ['Time Lags (seconds)','Correlation'])

    print("tlagg (s) ", tlagg, " rounded to .6f")
    return tlagg,fermi_lc_shifted,kw_lc,CC

In [2]:
def createNewTTE(oldTTEObj=None, tlagg=None, binsize=None,unit=None, obj=None, grb_ra=None, grb_dec=None):
    """
    Function preserves all the channels and the whole time range
    
    Create an EventList object from lists of times, channels, 
    and the channel bounds. The list of times and channels must be the 
    same length, and the list of channel boundaries are used to map the
    PHA index number in `pha_list` to energy channels.

    input------------
    oldTTeObj <TTE> 
    shiftedTTE <np.array>
    tlagg <float>: used to shift the trigtime
    binsize <float> : given in seconds. Need to convert spectrum to phaii object
    unit <str>: which detector is used
    obj <str> which GRB is observed
    grb_ra <float>
    grb_dec <float>
                             
    """
    phaii = oldTTEObj.to_phaii(bin_by_time,dt=binsize)
    spectrum = phaii.to_spectrum()
    newTTEdata = EventList.from_lists(times_list = oldTTEObj.data.times +tlagg, pha_list=oldTTEObj.data.pha,
                                      chan_lo=spectrum.lo_edges, chan_hi=spectrum.hi_edges)
    
    gti = [(shiftedTTE[0],shiftedTTE[-1])]
    newTrigtime = tte.trigtime + tlagg
    
    newTTEObj = TTE.from_data(data=newTTEdata,gti=gti,trigtime=newTrigtime, 
                              detector=unit,object=obj, ra_obj=grb_ra, dec_obj=grb_dec)
    print("new TTE object created")
    return newTTEObj
    

In [ ]:
def shiftTTE(tte=None,tlagg=None):
    """ Shift the fermi tte by a tshift
    input---------------
    tte <np.array>: List of photon arrival times
    tshift <float>: value to shift the tte by
    
    return----------
    shifted_tte <np.array>: tte shifted by tshift
    """
    shiftedTTE = tte + tlagg
    print("tte shifted by {}".format(tlagg))
    return shiftedTTE

In [ ]:
print("FERMI functions loaded successfully!")